In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

# import some message constructors
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Korean"),
  AIMessage(content="Hi, my nick name is Liberty!"),
  HumanMessage(content="남한과 북한의 거리는 얼마나되니? 그리고 네 이름은 무엇이니?")
]

chat.predict_messages(messages)

AIMessage(content='남한과 북한 사이의 거리는 가장 짧은 부분에서 약 50km 정도입니다. 제 이름은 립티입니다.')